In [1]:
# ✅ 셀 0: Chrome 및 Chromedriver 명시적 설치 (Colab에서 안정적 실행용)
!apt-get update -qq > /dev/null
!apt-get install -yqq unzip > /dev/null

# 기존 드라이버 제거
!rm -f /usr/bin/chromedriver

# Chrome 설치
!wget -q -O /tmp/chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i /tmp/chrome.deb > /dev/null 2>&1
!apt-get -f install -y > /dev/null

# Chromedriver 136 설치 (Chrome과 버전 일치)
!wget -q -O /tmp/chromedriver.zip https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.113/linux64/chromedriver-linux64.zip
!unzip -o /tmp/chromedriver.zip -d /tmp/chromedriver/ > /dev/null
!mv /tmp/chromedriver/chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver



������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
'rm'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
������ ��θ� ã�� �� �����ϴ�.
'mv'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'chmod'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
# 필요한 라이브러리 설치
!pip install -q selenium
!pip install -q webdriver-manager
!pip install -q beautifulsoup4


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')



# 경로 설정
save_path = 'D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트'
cookie_path = f'{save_path}/exolyt_login_state.json'



---



###선행되어야 되는 작업
###로그인 상태 쿠키 업로드

1. 먼저 로컬에 저장해놨던 example.py (playwright 코드로 로그 정보 json파일에 저장하는 코드) 실행
2. 드라이브에 새로 만들어진 파일(exolyt_login_state.json)로 업데이트

In [5]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 브라우저 옵션 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 ...")  # 실제 브라우저처럼 설정

# 웹드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.exolyt.com")  # 쿠키 설정 전 첫 접속

# 쿠키 파일 로드
with open('D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/exolyt_login_state.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

cookies_raw = json_data.get('cookies', [])

# ✅ 문자열로 저장된 경우 처리
cookies = []
for c in cookies_raw:
    if isinstance(c, str):
        try:
            cookies.append(json.loads(c))
        except Exception as e:
            print(f"❗ JSON 파싱 실패: {c[:30]}... → {e}")
    else:
        cookies.append(c)

# 쿠키 브라우저에 추가
for cookie in cookies:
    try:
        if 'domain' in cookie:
            cookie.pop('domain')  # 도메인 충돌 방지
        driver.add_cookie(cookie)
    except Exception as e:
        print(f"❗ 쿠키 추가 실패: {cookie.get('name', 'N/A')} → {e}")

# 재접속 후 인증 확인
driver.get("https://www.exolyt.com")
time.sleep(5)


In [6]:
driver.get("https://exolyt.com/user/tiktok/sontvn")
time.sleep(5)
print("📄 현재 페이지 제목:", driver.title)


📄 현재 페이지 제목: Exolyt - son


In [7]:
# Analytics 탭 클릭 및 로딩 페이지 기다리기

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Analytics 탭 클릭
try:
    analytics_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Analytics')]"))
    )
    analytics_button.click()
    print("✅ 'Analytics' 탭 클릭 완료")

    # 로딩 기다리기 (most liked hashtags 테이블 로드 대기)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//h3[contains(text(), 'Most liked hashtags')]"))
    )
    time.sleep(1)
except Exception as e:
    print("❌ 'Analytics' 탭 클릭 실패:", e)


✅ 'Analytics' 탭 클릭 완료


In [7]:
from bs4 import BeautifulSoup

# BeautifulSoup 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 'Most liked hashtags' 섹션 찾기
hashtag_heading = soup.find(lambda tag: tag.name == "h3" and "Most liked hashtags" in tag.get_text())

most_liked_hashtags = []
if hashtag_heading:
    current = hashtag_heading
    max_depth = 5
    while current and max_depth > 0:
        links = current.find_all("a", class_="ds-link")
        if links:
            most_liked_hashtags = [a.get_text(strip=True).lstrip("#") for a in links]
            break
        current = current.parent
        max_depth -= 1

# 결과 확인
if most_liked_hashtags:
    print("✅ Most liked hashtags:")
    for tag in most_liked_hashtags:
        print(tag)
else:
    print("❌ Most liked hashtags를 찾지 못했습니다.")


✅ Most liked hashtags:
foryou
fyp
sonsfam
billieeilish
hmhas
concert
viral
billieberlin
explore
hitmehardandsofttour
berlin
relatable
adele
hmhastour
capcut


In [8]:
import pandas as pd
from urllib.parse import urlparse

# username 정제
parsed_url = urlparse(driver.current_url)
path_parts = parsed_url.path.strip("/").split("/")
username = path_parts[-1] if path_parts else None

# 인증 여부 체크
verified = soup.find("svg", {"aria-label": "Verified"}) or soup.find(string="Verified")
is_verified = bool(verified)

# 해시태그 결합
hashtags_str = ", ".join(most_liked_hashtags)

# DataFrame 저장
result_df = pd.DataFrame([{
    "username": username,
    "verified": is_verified,
    "top_hashtags": hashtags_str
}])

# 결과 출력
print(result_df)


# CSV 저장
result_csv_path = f"{save_path}/exolyt_result.csv"
result_df.to_csv(result_csv_path, index=False, encoding='utf-8-sig')
print(f"📁 CSV 저장 완료: {result_csv_path}")


  username  verified                                       top_hashtags
0   sontvn      True  foryou, fyp, sonsfam, billieeilish, hmhas, con...
📁 CSV 저장 완료: /content/drive/MyDrive/DAT 팀_김동영/이윤빈/exolyt_result.csv


In [9]:
result_df.head(5)

,username,verified,top_hashtags
0,sontvn,True,"foryou, fyp, sonsfam, billieeilish, hmhas, con..."


### 모든 계정들 테스트

In [10]:
import pandas as pd

# 엑셀 파일 경로
handle_path = '/content/drive/MyDrive/DAT 팀_김동영/이윤빈/소셜핸들.xlsx'

# 파일에서 Username 열 읽기
handle_df = pd.read_excel(handle_path)

# @ 기호 제거하고 리스트로 변환
usernames = handle_df['Username'].dropna().apply(lambda x: x.lstrip('@')).tolist()

# 결과 확인 (상위 5개 미리보기)
print(usernames[:])




['theonlyhaven', 'turquoise.dec', 'ttalgi.ellie', 'srsaskin_', '_anjello', 'jf.skin24', 'cinthia_dosamples', 'lee.ly1012', 'prettywrinkles', 'xevelynluu', 'chloelouiseliveslife', 'melissahemmingsley', 'kbootyjenny', 'skincare.ninja', 'ugc_withjessi', 'so.calmama', 'jessicaroseugc', 'kayskwa', 'jennasiee', 'ugcxmichelle', 'autumnsaro', 'tinaangelinaaa', 'crystalschob3', 'life.w.lex4', 'jensnomz', 'srsdigitaldiary', 'eloise_ugc', 'shellllls_lifeeee', 'xokena_', 'mindfulugc', 'nooni.ugc', 'heyitsaliquinn', 'glo.ssary', 'ugc.karla', 'pretty.squirrelss', 'danielaapryl', 'allbriwoo', 'grw.alii_', 'jilliangottlieb', '_taniagrover', 'zhala.beauty', 'skinbykristin', 'lifewtamia', 'sunscreen.addict', 'toninlanaa', 'ft.angela', 'pinaydora', 'yangmediary', 'd_dudley4', 'beauty__bymegs', 'witandarte', 'beautycloudlove', 'psdiwan22', 'auraofmahmuda', 'vickyoffishal', '__allaboutskincare__', 'klymi.na', 'jeracoolio', 'razan.alani', 'angelinajarboe', 'waves_for_dayzz', 'jmeans77', 'carmelk1', 'ouibutn

In [11]:
print(handle_df.shape)

(305, 1)


In [16]:
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


In [17]:
# 크롤링 결과 저장용 리스트
results = []

# ✅ 사용자마다 driver 재시작
for username in usernames:
    print(f"\n🚀 수집 시작: {username}")
    try:
        # 1️⃣ 드라이버 실행
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(f"https://exolyt.com/user/tiktok/{username}")
        time.sleep(random.uniform(4, 6))

        # 2️⃣ Analytics 탭 클릭
        try:
            WebDriverWait(driver, 10).until(
                EC.invisibility_of_element_located((By.CLASS_NAME, "ds-fade-in"))
            )
            analytics_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Analytics')]"))
            )
            driver.execute_script("arguments[0].click();", analytics_button)
            print("✅ 'Analytics' 탭 클릭 완료")

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//h3[contains(text(), 'Most liked hashtags')]"))
            )
            time.sleep(1)
        except Exception as e:
            print(f"❌ {username} - Analytics 탭 클릭 실패: {e}")
            driver.quit()
            continue

        # 3️⃣ BeautifulSoup 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 인증 여부 확인
        verified = soup.find("svg", {"aria-label": "Verified"}) or soup.find(string="Verified")
        is_verified = bool(verified)

        # 해시태그 추출
        most_liked_hashtags = []
        hashtag_heading = soup.find(lambda tag: tag.name == "h3" and "Most liked hashtags" in tag.get_text())
        if hashtag_heading:
            current = hashtag_heading
            max_depth = 5
            while current and max_depth > 0:
                links = current.find_all("a", class_="ds-link")
                if links:
                    most_liked_hashtags = [a.get_text(strip=True).lstrip("#") for a in links]
                    break
                current = current.parent
                max_depth -= 1

        # 결과 저장
        results.append({
            "username": username,
            "verified": is_verified,
            "top_hashtags": ", ".join(most_liked_hashtags)
        })
        print(f"✅ {username} - 수집 성공")

    except Exception as e:
        print(f"❌ {username} - 처리 중 오류 발생: {e}")

    finally:
        driver.quit()  # 4️⃣ 드라이버 종료

# 🔚 최종 결과 저장
result_df = pd.DataFrame(results)
result_csv_path = f"{save_path}/exolyt_result.csv"
result_df.to_csv(result_csv_path, index=False, encoding='utf-8-sig')
print(f"\n📁 CSV 저장 완료: {result_csv_path}")



🚀 수집 시작: theonlyhaven


❌ theonlyhaven - Analytics 탭 클릭 실패: Message: 
Stacktrace:
#0 0x55a8b3c3f71a <unknown>
#1 0x55a8b36e20a0 <unknown>
#2 0x55a8b37339b0 <unknown>
#3 0x55a8b3733ba1 <unknown>
#4 0x55a8b3781ea4 <unknown>
#5 0x55a8b37593cd <unknown>
#6 0x55a8b377f2a0 <unknown>
#7 0x55a8b3759173 <unknown>
#8 0x55a8b3725d4b <unknown>
#9 0x55a8b37269b1 <unknown>
#10 0x55a8b3c048cb <unknown>
#11 0x55a8b3c087ca <unknown>
#12 0x55a8b3bec622 <unknown>
#13 0x55a8b3c09354 <unknown>
#14 0x55a8b3bd145f <unknown>
#15 0x55a8b3c2d4f8 <unknown>
#16 0x55a8b3c2d6d6 <unknown>
#17 0x55a8b3c3e586 <unknown>
#18 0x7ab58da07ac3 <unknown>


🚀 수집 시작: turquoise.dec


❌ turquoise.dec - Analytics 탭 클릭 실패: Message: 
Stacktrace:
#0 0x5c2bab22a71a <unknown>
#1 0x5c2baaccd0a0 <unknown>
#2 0x5c2baad1e9b0 <unknown>
#3 0x5c2baad1eba1 <unknown>
#4 0x5c2baad6cea4 <unknown>
#5 0x5c2baad443cd <unknown>
#6 0x5c2baad6a2a0 <unknown>
#7 0x5c2baad44173 <unknown>
#8 0x5c2baad10d4b <unknown>
#9 0x5c2baad119b1 <unknown>
#10 0x5c2bab1ef8cb <unknown>
#11 0x5c2bab1f37ca <unknown>
#12 0x5c2bab1d7622 <unknown>
#13 0x5c2bab1f4354 <unknown>
#14 0x5c2bab1bc45f <unknown>
#15 0x5c2bab2184f8 <unknown>
#16 0x5c2bab2186d6 <unknown>
#17 0x5c2bab229586 <unknown>
#18 0x78ebe092cac3 <unknown>


🚀 수집 시작: ttalgi.ellie


KeyboardInterrupt: 

### most liked hashtags가 들어있는 페이지의 HTML 구조 확인하는 코드들 (이젠 필요없음, 참고만)

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 'Analytics' 탭 클릭 (버튼의 텍스트로 찾음)
try:
    analytics_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Analytics')]"))
    )
    analytics_button.click()
    print("✅ 'Analytics' 탭 클릭 완료")
    time.sleep(3)  # 탭 클릭 후 로딩 대기
except Exception as e:
    print("❌ 'Analytics' 탭 클릭 실패:", e)

# BeautifulSoup로 다시 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 'Most liked hashtags' 문구가 포함된 h3 태그 탐색
hashtag_heading = soup.find(lambda tag: tag.name == "h3" and "Most liked hashtags" in tag.get_text())

if hashtag_heading:
    parent_div = hashtag_heading.find_parent()
    print("✅ Most liked hashtags 전체 HTML 구조:")
    print(parent_div.prettify())
else:
    print("❌ 여전히 'Most liked hashtags' 섹션을 찾을 수 없습니다.")



In [ ]:
# 'Analytics' 탭은 이미 눌렀다고 가정
soup = BeautifulSoup(driver.page_source, "html.parser")

# 1. 'Most liked hashtags' 제목이 있는 h3 태그 찾기
hashtag_heading = soup.find(lambda tag: tag.name == "h3" and "Most liked hashtags" in tag.get_text())

# 2. 이 제목에서 시작해서 a.ds-link 태그가 들어있는 전체 섹션 찾기
if hashtag_heading:
    # h3 기준으로 a 태그가 나올 때까지 부모 구조 탐색
    current = hashtag_heading
    max_depth = 5  # 너무 멀어지지 않도록 제한
    while current and max_depth > 0:
        links = current.find_all("a", class_="ds-link")
        if links:
            print("✅ 해시태그 a 태그 포함된 전체 섹션 HTML:")
            print(current.prettify())
            break
        current = current.parent
        max_depth -= 1
    else:
        print("❌ a.ds-link 태그를 포함한 섹션을 찾지 못했습니다.")
else:
    print("❌ 'Most liked hashtags' 제목이 있는 h3 태그를 찾을 수 없습니다.")
